# Notes

- jar file = Java ARchive. file format based on ZIP used for aggregating many files into one
- Biggest issue was setting up BQ connector - needed a compatible .jar file according to scala version - https://github.com/GoogleCloudDataproc/spark-bigquery-connector

- To submit a job directly to the cluster
    - $ gcloud dataproc jobs submit spark --jar gs://spark-lib/bigquery/spark-2.4-bigquery-0.28.0-preview.jar my_script.py --cluster clustertest2 --region europe-west1 
    - $ gcloud dataproc jobs submit spark --help
    
- SparkSession.builder.appName() did not work after a while and had to totally restart the cluster for some reason to get working again

In [1]:
# !pip install koalas # need to install if spark version < 3.2

from pyspark.sql import SparkSession
import databricks.koalas as ks
import pandas as pd

In [2]:
# 1.5.80-debian10
# https://cloud.google.com/dataproc/docs/concepts/versioning/dataproc-release-1.5 -- shows that it has spark version 2.4.8 
!scala -version
# think we want spark version of >3.2 - should be available with 2.0-centos8
!spark-sql --version # to get spark version

Scala code runner version 2.12.10 -- Copyright 2002-2019, LAMP/EPFL and Lightbend, Inc.
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 2.4.8
      /_/
                        
Using Scala version 2.12.10, OpenJDK 64-Bit Server VM, 1.8.0_352
Branch 
Compiled by user  on 2023-01-03T22:09:32Z
Revision 
Url 
Type --help for more information.


__To create dataproc cluster__

In [ ]:
# conf = SparkConf()
# conf.setMaster('spark://spark-master-svc.default.svc.cluster.local:7077')
# conf.setAppName('spark-basic')
# conf.set("spark.driver.host", "jupyter-headless.default.svc.cluster.local")
# sc = SparkContext(conf=conf)

In [3]:
# suddenly taking a lot longer... not sure why # 
spark = SparkSession.builder.appName('spark-bigquery-demo').config('spark.jars','gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar').getOrCreate()
# will output the results of DataFrames in each step without the new need to show df.show() and also improves the formatting of the output #
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)


In [25]:
# table = "bigquery-public-data.wikipedia.pageviews_2020"

spark_df = spark.read.format("bigquery").option("table", 'sky-uk-ids-analytics-prod.uk_poc_beam_ic.BBDB_140k').load()
 
#     .option('header', 'true') \
#     .option('delimiter', ',') \
#     .option('inferSchema', 'true')
# spark_df.printSchema()

In [ ]:
# # Setup hadoop fs configuration for schema gs://
# conf = spark.sparkContext._jsc.hadoopConfiguration()
# conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
# conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

# df.createOrReplaceTempView("words")

# csv_file_path = 'gs://YOUR_BUCKET/test.json'
# df = spark.read.json(json_file_path, schema, multiLine=True)

In [23]:
# If want to query this table directly from here : # 
spark_df.createOrReplaceTempView("tempview")
count = spark.sql("SELECT count(1) FROM tempview WHERE date_ = '2022-10-28' ")
count.show()

### Saving the data to BigQuery
# count.write.format('bigquery').option('table', 'sky-uk-ids-analytics-prod.uk_poc_beam_ic.count').save()

+--------+
|count(1)|
+--------+
|   67286|
+--------+



In [ ]:

train_df = spark_df = spark.read.format("bigquery").option("table", 'project.uk_poc_beam_ic.Baseline_model_joint_UNSAMPLED') \
  .load()
self.val_df = (self.client.query(dev_query.format(columns = self.parameters_file.COLUMNS,
                                  table = self.dev_dataset_name,
                                  identity_column = self.parameters_file.IDENTITY_COLUMN,
                                  splitter = self.parameters_file.VALIDATION_SPLIT))
                                  .result()
                                  .to_dataframe())

self.test_df = (self.client.query(dev_query.format(columns = self.parameters_file.COLUMNS,
                                  table = self.dev_dataset_name,
                                  identity_column = self.parameters_file.IDENTITY_COLUMN,
                                  splitter = self.parameters_file.TEST_SPLIT))
                                  .result()
                                  .to_dataframe())

self.oos_df = (self.client.query(oos_query.format(columns = self.parameters_file.COLUMNS,
                                  table = self.oos_dataset_name,
                                  identity_column = self.parameters_file.IDENTITY_COLUMN,
                                  splitter = self.parameters_file.OOS_SPLIT))
                                  .result()
                                  .to_dataframe())

__Convert to a koalas dataframe__

In [ ]:
ks.set_option('compute.default_index_type', 'distributed') # Need to set this OR 'distributed-sequence' to speed up .head() 

# Create a Koalas DataFrame from pandas DataFrame
# df_ = ks.from_pandas(df)

kdf = spark_df.to_koalas(index_col=['account_number','date_']) # index_col might speed it up 


: 